• Image Stitching

• Video Stabilization

In [24]:
import cv2
import numpy as np

In [4]:
# Detect and match features using knnMatch

def getKnnMatches(img_query, img_train, ratio, direction='right', showMatches=True):

    # detect keypoints and compute descriptors using ORB
    orb = cv2.ORB_create(nfeatures=2000)
    kp_query, dsc_query = orb.detectAndCompute(img_query, None) # left image
    kp_train, dsc_train = orb.detectAndCompute(img_train, None) # right image

    # get matches on the two images
    bf = cv2.BFMatcher_create()
    if direction == 'right':
        matches = bf.knnMatch(dsc_query, dsc_train, k=2)
    if direction == 'left':
        matches = bf.knnMatch(dsc_train, dsc_query, k=2)
    
    # appy ratio test
    good = []
    for m, n in matches:
        if m.distance < ratio*n.distance:
            good.append([m])

    if showMatches:
        # draw matches
        if direction == 'right':
            img_matches = cv2.drawMatchesKnn(img_query, kp_query, img_train, kp_train, good, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)    

        if direction == 'left':
            img_matches = cv2.drawMatchesKnn(img_train, kp_train, img_query, kp_query, good, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)    
        cv2.imshow("matches", img_matches)
        cv2.waitKey()
    
    if direction == 'right':
        return kp_query, kp_train, good
    
    if direction == 'left':
        return kp_train, kp_query, good

    
    

In [5]:
# stitch left image to right image (right is anchored image)
def stitch_left(img_left, img_right, ratio, showMatches=True):

    cv2.imshow("left image", img_left)
    cv2.imshow("right image", img_right)
    cv2.waitKey()

    # get matches and keypoints of query image (right) in train image (left)
    kp_right, kp_left, good = getKnnMatches(img_right, img_left, ratio, 'left')


    # we need at least four matches to find homography between the images
    if len(good) > 4:
        
        # extract locations (keypoints) of good matches
        p_left, p_right = [], []
        p_right = np.float32([kp_right[m[0].queryIdx].pt for m in good])
        p_left = np.float32([kp_left[m[0].trainIdx].pt for m in good])
        p_right = np.asarray(p_right)
        p_left = np.asarray(p_left)

        # find homography using RANSAC (left image is src plane, right is target plane)
        H, status = cv2.findHomography(p_left, p_right, cv2.RANSAC)
        H_inv = np.linalg.inv(H)
        print("Inverse Homography:\n", H_inv)

        # calculate positions of warped corners of left image by matrix-vector product
        #  p_00 ------------ p_c0
        #   |                 |
        #  p_0r ------------ p_cr
        h = img_left.shape[0]
        w = img_left.shape[1]
        p_00 = np.dot(H_inv, np.array([0,0,1])) 
        p_00 = p_00/p_00[-1] # homogenous to cartesian coordinates x=x/w, y=y/w
        p_0r = np.dot(H_inv, np.array([0, h, 1])) 
        p_0r = p_0r/p_0r[-1] 
        p_c0 = np.dot(H_inv, np.array([w, 0, 1])) 
        p_c0 = p_c0/p_c0[-1] 
        p_cr = np.dot(H_inv, np.array([w, h, 1])) 
        p_cr = p_cr/p_cr[-1] 

        # determine maximum in x and y direction
        max_x = int(max([0, w, p_00[0], p_0r[0], p_c0[0], p_cr[0]]))
        max_y = int(max([0, h, p_00[1], p_0r[1], p_c0[1], p_cr[1]]))

        # determine minimum in x and y direction
        offset_x = abs(int(min([0, w, p_00[0], p_0r[0], p_c0[0], p_cr[0]])))
        offset_y = abs(int(min([0, h, p_00[1], p_0r[1], p_c0[1], p_cr[1]])))

        # determine resulted size of warped image
        dsize = (max_x + offset_x, max_y + offset_y)
        print("image dsize =>", dsize)

        # add translation (offset) to inverse H 
        # => if warped corners are outside the image, we need translation
        # H_inv_00, H_inv_01, H_inv_02 => tx=H_inv_02
        # H_inv_10, H_inv_11, H_inv_12 => ty=H_inv_12
        # H_inv_20, H_inv_21, H_inv_22
        H_inv[0][-1] += offset_x # iH[0][-1]: last item of 1st row
        H_inv[1][-1] += offset_y # iH[1][-1]: last item of 2nd row


        # warp left image by inverse H and calulated size
        output = cv2.warpPerspective(img_left, H_inv, dsize)
        cv2.imshow("warped image", output)
        cv2.waitKey()

        # add right image
        output[offset_y:img_right.shape[0] + offset_y, offset_x:img_right.shape[1] + offset_x] = img_right

        return output
    else:
        print("\nError: not enough matches\n")
        return None



In [6]:
# stitch right image to left image (left is anchored image)
def stitch_right(img_left, img_right, ratio, showMatches=True):

    cv2.imshow("left image", img_left)
    cv2.imshow("right image", img_right)
    cv2.waitKey()

    # get matches and keypoints of query image (left) in train image (right)
    kp_left, kp_right, good = getKnnMatches(img_left, img_right, ratio, 'right')

    # we need at least four matches to find homography between the images
    if len(good) > 4:
        
        # extract locations (keypoints) of good matches
        p_left, p_right = [], []
        p_left = np.float32([kp_left[m[0].queryIdx].pt for m in good])
        p_right = np.float32([kp_right[m[0].trainIdx].pt for m in good])
        p_left = np.asarray(p_left)
        p_right = np.asarray(p_right)
        
        # find homography using RANSAC (right image is src plane, left is target plane)
        H, status = cv2.findHomography(p_right, p_left, cv2.RANSAC)        
        
        # calculate positions of warped corners of right image by matrix-vector product
        #  p_00 ------------ p_c0
        #   |                 |
        #  p_0r ------------ p_cr
        h = img_right.shape[0]
        w = img_right.shape[1]
        p_00 = np.dot(H, np.array([0,0,1])) 
        p_00 = p_00/p_00[-1] # homogenous to cartesian coordinates x=x/w, y=y/w
        p_0r = np.dot(H, np.array([0, h, 1])) 
        p_0r = p_0r/p_0r[-1] 
        p_c0 = np.dot(H, np.array([w, 0, 1])) 
        p_c0 = p_c0/p_c0[-1] 
        p_cr = np.dot(H, np.array([w, h, 1])) 
        p_cr = p_cr/p_cr[-1] 
        # determine maximum in x and y direction
        max_x = int(max([0, img_left.shape[1], p_00[0], p_0r[0], p_c0[0], p_cr[0]]))
        max_y = int(max([0, img_left.shape[0], p_00[1], p_0r[1], p_c0[1], p_cr[1]]))
        # determine minimum in x and y direction
        offset_x = abs(int(min([0, img_left.shape[1], p_00[0], p_0r[0], p_c0[0], p_cr[0]])))
        offset_y = abs(int(min([0, img_left.shape[0], p_00[1], p_0r[1], p_c0[1], p_cr[1]])))
        # determine resulted size of warped image
        dsize = (max_x + offset_x, max_y + offset_y)
        print("image dsize =>", dsize)
        
        # add translation (offset) to inverse H 
        # => if warped corners are outside the image, we need translation
        # H_00, H_01, H_02 => tx=H_02
        # H_10, H_11, H_12 => ty=H_12
        # H_20, H_21, H_22
        H[0][-1] += offset_x # iH[0][-1]: last item of 1st row
        H[1][-1] += offset_y # iH[1][-1]: last item of 2nd row
        
        # perspective transformation of right image using computed homography
        output = cv2.warpPerspective(img_right, H, dsize)
        cv2.imshow("warped image", output)
        cv2.waitKey()
        
        # add left image
        output[offset_y:img_left.shape[0] + offset_y, offset_x:img_left.shape[1] + offset_x] = img_left
        
        return output
    else:
        print("\nError: not enough matches\n")
        return None

In [7]:
# load image
img_left = cv2.imread('Image/01.jpg')
img_right = cv2.imread('Image/02.jpg')
path = 'Image/'

# # stitch images
# direction = 'right'
# if direction=='right': # stitch images (from right to left)
#     result = stitch_right(img_left, img_right, 0.6)
# if direction=='left': # stitch images (from left to right)
#     result = stitch_left(img_left, img_right, 0.7)

# if result is not None:
#     # write panorama
#     cv2.imwrite(path +direction+"_2_panorama.jpg", result)

#     # show panorama
#     # resize image since cv2.imshow in python has problems to show big images
#     scale = 70
#     dim = int(result.shape[1] * scale / 100), int(result.shape[0] * scale / 100)
#     pano = cv2.resize(result, dim) 
#     cv2.imshow("stitched images", pano)
#     cv2.waitKey(0)

cv2.destroyAllWindows()



In [11]:
# load and optionally resize images
N = 3
path = 'Image/'
filename_prefix = '0'
images = []

# traversing over images through loop
for i in range(N):
    img = cv2.imread(path+filename_prefix+str(i+1)+'.jpg')
    images.append(img)

# iterate through the images to stitch by given direction
direction = 'center'
panorama = images[0]
if direction=='left': 
    for i in range(1,N):
        panorama = stitch_left(panorama, images[i], 0.7)

if direction=='right': 
    for i in range(1,N):
        panorama = stitch_right(panorama, images[i], 0.7)

if direction=='center':
    for i in range(1,N):
        if i <= int(N/2) :
            panorama = stitch_left(panorama, images[i], 0.7)
        else:
            panorama = stitch_right(panorama, images[i], 0.7)

if panorama is not None:
    # write panorama
    cv2.imwrite(path+direction+"_"+str(i+1)+"_panorama.jpg", panorama)

    # show panorama
    # resize image since cv2.imshow in python has problems to show big images
    scale = 70
    dim = int(panorama.shape[1] * scale / 100), int(panorama.shape[0] * scale / 100)
    panorama = cv2.resize(panorama, dim) 
    cv2.imshow("Panorama", panorama)
    cv2.waitKey(0)

cv2.destroyAllWindows()


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/sudhir/.local/lib/python3.10/site-packages/cv2/qt/plugins"


Inverse Homography:
 [[ 1.04461640e+00 -7.90827336e-02 -5.00873092e+02]
 [ 7.91160226e-02  9.73513755e-01 -1.19936026e+01]
 [ 2.40605319e-04 -1.21149686e-04  8.81908364e-01]]
image dsize => (1294, 565)
image dsize => (1711, 565)


In [22]:
# Stitching using OpenCV
#print("Enter the number of images:")
#nimg = int(input())

# load and optionally resize images
N = 3
path = 'Image/'
filename_prefix = '0'
scale_percent = 80 # percentage of original size
images = []

# traversing over images through loop
for i in range(N):
    name = path+filename_prefix+str(i+1)+'.jpg'
    # print(name)
    img = cv2.imread(name)
    # print(img.shape)
    if scale_percent != 100:
        dim = int(img.shape[1] * scale_percent / 100), int(img.shape[0] * scale_percent / 100)
        img = cv2.resize(img, dim)

    images.append(img)

# initialize OpenCV image sticher object and then perform image stitching
stitcher = cv2.Stitcher.create()
(status, stitched) = stitcher.stitch(images)

# if the status is '0', then OpenCV successfully performed image stitching
if status == 0:
	# display images
	for i in range(N):
		cv2.imshow("Input Image "+str(i+1), images[i])
  
	cv2.imshow("Stitched Image", stitched)
	cv2.waitKey(0)

# otherwise the stitching failed, e.g. not enough keypoints being detected
else:
	print("image stitching failed ({})".format(status))
	
cv2.destroyAllWindows()


Video Stabilization:

Find Motion between frame
Find good features in current and previous frame
Find transformation by set of keypoints to map previous frame to current frame
Store rotation and translation values
Sum up values to create trajectory
Smooth trajectory by average filter
Calculate smooth motion between frame
Apply smoothed camera motion to each frame

In [31]:
def averageFiltering(path, ksize): 
    # define average filter 
    window_size = 2*ksize+1
    filter = np.ones(window_size)/window_size 

    # add padding to the boundaries ('edge': repeat edge value)
    path_pad = np.pad(path, (ksize, ksize), 'edge') 

    # apply convolution to given trajectory
    path_smoothed = np.convolve(path_pad, filter, mode='same') 

    # return smoothed trajectory without padding
    return path_smoothed[ksize:-ksize] 

# read video file
path = 'Video/'
cp = cv2.VideoCapture(path+'video.mp4')

# get number of frames
n_frames = int(cp.get(cv2.CAP_PROP_FRAME_COUNT))
print("number of frame:", n_frames)

# get size of frame
width = int(cp.get(cv2.CAP_PROP_FRAME_WIDTH)) 
height = int(cp.get(cv2.CAP_PROP_FRAME_HEIGHT))
print("height:", width)
print("height:", height)

# read first frame and convert to grayscale
_, prev = cp.read()
prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)


number of frame: 797
height: 480
height: 272


In [32]:
##################################################
# find camera motions between consequent frames: #
##################################################

# define array to hold translation (dx,dy) and rotation angle of each frame
transforms = np.zeros((n_frames-1, 3), np.float32) 

for i in range(n_frames-2):
    # find most prominent corners in previous frame by Shi-Tomasi detector
    prev_pts = cv2.goodFeaturesToTrack(prev_gray, maxCorners=200, qualityLevel=0.01, minDistance=30, blockSize=3)

    # read next frame
    succ, frame = cp.read()

    # convert frame to grayscale
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # track feature points in current frame by optical flow
    frame_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prev_pts, None)

    # estimate affine transformation between keypoints of previous and current frame
    mat, inlier = cv2.estimateAffine2D(prev_pts, frame_pts) 

    # extract translation an rotation
    dx = mat[0,2]
    dy = mat[1,2]
    angle = np.arctan2(mat[1,0], mat[0,0])
    
    # save translation and rotation angle
    transforms[i] = [dx,dy,angle] 
    
    # current frame will be the next previous frame
    prev_gray = frame_gray

# create trajectory by cumulative sum of transform for each dx,dy and angle
trajectory = np.cumsum(transforms, axis=0) #axis=0: sum over rows for each of the 3 columns

# smooth trajectory by applying average filter to x, y and angle 
smoothed_trajectory = np.copy(trajectory) 
ksize = 9
for i in range(3):
    smoothed_trajectory[:,i] = averageFiltering(trajectory[:,i], ksize)

# take the difference to obtain smoothed trajectory
difference = smoothed_trajectory - trajectory
transforms_smooth = transforms + difference

In [33]:
###############################################
# apply smoothed camera motion to each frame: #
###############################################
# reset stream to first frame 
cp.set(cv2.CAP_PROP_POS_FRAMES, 0) 
for i in range(n_frames-2):

    # read next frame
    success, frame = cp.read() 

    # extract smoothed translations and angle
    dx = transforms_smooth[i,0]
    dy = transforms_smooth[i,1]
    angle = transforms_smooth[i,2]

    # set up transformation matrix by extracted values
    m = np.zeros((2,3), np.float32)
    m[0,0] = np.cos(angle)
    m[0,1] = -np.sin(angle)
    m[1,0] = np.sin(angle)
    m[1,1] = np.cos(angle)
    m[0,2] = dx
    m[1,2] = dy

    # apply affine warping to the given frame
    frame_stabilized = cv2.warpAffine(frame, m, (width,height))

    # concate before and after frame for comparing purpose
    result = cv2.hconcat([frame, frame_stabilized])
        
    # show concated frames 
    cv2.imshow("Video Stabilization", result)
    cv2.waitKey(10)

cp.release() # release video
cv2.destroyAllWindows()